
## Inertial Measurement Unit (IMU)

An IMU is a device that uses accelerometers and gyroscopes to measure
forces (via accelerations) and angular rates acting on a body. The IMU
on the Duckiedrone is a built-in component of the flight controller. Data
provided by the IMU are used by the state estimator, which you will be
implementing in the next project, to better understand its motion in
flight. In addition, the flight controller uses the IMU data to
stabilize the drone from small perturbations.

The IMU can be used to measure global orientation of roll and pitch, but **not** yaw.  This is because it measures acceleration due to gravity, so it can measure the downward pointing gravity
vector.  However, this information does not give a global yaw measurement.  Many
drones additionally include a magnetometer to measure global yaw
according to the Earth's magnetic field, but the Duckiedrone does not
have this sensor.

Note that IMUs *do NOT* measure position or linear velocity.  The
acceleration measurements can be integrated (added up over time) to
measure linear velocity, and these velocity estimates can be
integrated again to measure position.  However, without some absolute
measurement of position or velocity, these estimates will quickly
diverge.  To measures these properties of the drone, we need to use
the camera as described below.



(sensor-assignment-quaternion)=
# Part 3: Gimbal Lock

The orientation of an object in 3D space can be described by a set of three values: $(\alpha, \beta, \gamma)$, where $\alpha$ is roll, $\beta$ is pitch, and $\gamma$ is yaw. 

<!-- <figure>
  <div style="text-align:center;">
  <figcaption>Roll, pitch, and yaw</figcaption>
  <img style='width:250px' src='../../_images/sensors/airplane_roll_pitch_yaw.png' />
</figure>
 -->
```{figure} ../../_images/sensors/airplane_roll_pitch_yaw.png

Roll, pitch, and yaw
```

Mathematically, any point $\mathbf{p}$ on an object that undergoes rotation $(\alpha, \beta, \gamma)$ will have a new coordinate $\mathbf{p'}$ calculated as follows:

$$
\mathbf{p'} = R \mathbf{p}
$$

Where:

$$
\begin{align*}
    \mathbf{p'} &= \begin{bmatrix}
  x' \\
  y' \\
  z' \\
    \end{bmatrix} \\
    R &=    
    \begin{bmatrix}
  1 & 0 & 0 \\
  0 & \cos \alpha & -\sin \alpha \\
  0 & \sin \alpha & \cos \alpha \\
    \end{bmatrix}
    \begin{bmatrix}
  \cos \beta & 0 & \sin \beta \\
  0 & 1 & 0 \\
  -\sin \beta & 0 & \cos \beta \\
    \end{bmatrix}
    \begin{bmatrix}
  \cos \gamma & -\sin \gamma & 0 \\
  \sin \gamma & \cos \gamma & 0 \\
  0 & 0 & 1 \\
    \end{bmatrix} \\
    \mathbf{p} &= \begin{bmatrix}
  x \\
  y \\
  z \\
    \end{bmatrix}
\end{align*}
$$

Ideally, we would hope that the parameters $(\alpha, \beta, \gamma)$ are enough to rotate any point $\mathbf{p}$ (distance $d$ from the origin) to any other point $\mathbf{p'}$ (also distance $d$ from the origin, since rotations do not change distance). Upon closer thought, it would seem as if we have more than enough parameters to do this, since it only takes two parameters $(\theta, \phi)$ to describe all points on the 3D unit sphere

<!-- <figure>
  <div style="text-align:center;">
  <figcaption>Two parameters sweeping out a sphere</figcaption>
  <img style='width:250px' src='../../_images/sensors/3d_unit_sphere.jpg' />
</figure> -->

```{figure} ../../_images/sensors/3d_unit_sphere.jpg

Two parameters sweeping out a sphere
```

However, this intuition is a bit off. If any one parameter is held fixed, it may be impossible for $\mathbf{p}$ to be rotated to some other $\mathbf{p'}$ by varying the remaining two parameters. Moreover, if a certain parameter is set to a certain problematic value, then varying the remaining two parameters will either sweep out a circle (not a sphere!), or not affect $\mathbf{p}$ at all, depending on what $\mathbf{p}$ is. This result is way different from what we expected! The name for this degenerate case is gimbal lock. 

## Questions


1. Suppose an airplane increases its pitch to $\pi / 2$ (i.e. $90 \deg$ ):

    ```{figure} ../../_images/sensors/airplane_pitch_0.jpg

    Airplane
    ```

    ```{figure} ../../_images/sensors/airplane_pitch_90.jpg

    Airplane with pitch at 90 degrees
    ```

    <!-- 
    <figure>
      <div style="text-align:center;">
      <figcaption>Airplane</figcaption>
      <img style='width:250px' src='../../_images/sensors/airplane_pitch_0.jpg' />
    </figure>

    <figure>
      <div style="text-align:center;">
      <figcaption>Airplane with pitch at 90 degrees</figcaption>
      <img style='width:250px' src='../../_images/sensors/airplane_pitch_90.jpg' />
    </figure>
     -->

    Let $R_{\text{gim} \beta}$ denote the rotation matrix $R$ for $\beta = \pi / 2$. Prove that

    $$
    R_{\text{gim} \beta} = 
        \begin{bmatrix}
      0 & 0 & 1 \\
      \sin(\alpha + \gamma) & \cos(\alpha + \gamma) & 0 \\
      -\cos(\alpha + \gamma) & \sin(\alpha + \gamma) & 0 \\
        \end{bmatrix}
    $$ 


1. Consider the point $\mathbf{p} = [0 \quad 1 \quad 0]^T$ on the pitched airplane, i.e. the tip of the wing. Does there exist any $\alpha, \gamma$ such that $\mathbf{p'} = R_{\text{gim} \beta} \mathbf{p}$ for $\mathbf{p'} = [1 \quad 0 \quad 0]^T$?
  
    Show your work and briefly explain your reasoning (1-2 sentences). 


1. Consider the point $\mathbf{p} = [0 \quad 1 \quad 0]^T$ on the pitched airplane, i.e. the tip of the wing:
    Can we set $\alpha, \gamma$ such that:

    $$
    \mathbf{p'} = R_{\text{gim} \beta} \mathbf{p}
    $$

    for some $\mathbf{p'}$ on the XY unit circle (e.g. $[\frac{\sqrt 2}{2} \quad \frac{\sqrt 2}{2} \quad 0]^T$)?
    
    You do not have to show any work, but briefly explain your reasoning (1-2 sentences). 


1. Consider the point $\mathbf{p} = [0 \quad 1 \quad 0]^T$ on the pitched airplane, i.e. the tip of the wing.
    Can we set $\alpha, \gamma$ such that:

    $$
    \mathbf{p'} = R_{\text{gim} \beta} \mathbf{p}
    $$
    
    for some $\mathbf{p'}$ on the YZ unit circle (e.g. $[0 \quad \frac{\sqrt 2}{2} \quad \frac{\sqrt 2}{2}]^T$)?
  
    You do not have to show any work, but briefly explain your reasoning (1-2 sentences). 


Reflect on your answers to the previous 4 questions. What are the questions trying to portray? Why are the answers different? Why is $\pi / 2$ (i.e. $90 \deg)$ a "certain problematic value"? What would happen to an airplane that pitched that much? Could a pilot recover from such a situation? Are 3 parameters enough to allow for rotations in all situations? 




(sensors-assignment-imu)=
# Interfacing with the IMU

Your drone is equipped with a [Skyline32 Flight Controller](https://hobbyking.com/en_us/skyline32-acro-flight-controller-w-baseflight-cleanflight.html) which has a built in IMU. In this part of the project, you will learn how to interface with the flight controller board to extract the attitude, accelerations, angular rates of the drone from the built-in IMU. In addition, you will extract the battery levels from the flight controller so that you'll be able to tell when you're battery is too low.

**Setup**
Change directories into `~/ws/src/pidrone_pkg` and modify `pi.screenrc` to start up with your flight controller node by changing `python flight_controller_node.py\n` to `rosrun project-sensors-yourGithubName student_flight_controller_node.py\n` (or, alternatively, `python \path\to\student_flight_controller_node.py\n`).

## Problem 1: Extracting the Battery Data
The flight controller is capable of reading the voltage and current of the power source plugged into the drone. This is possible because of the red and brown wire pair (i.e. _battery monitor_ wire pair) plugged into the FC. The power information is useful because it allows us to programmatically shut down the drone if the voltage is too low (e.g. LiPo batteries are quickly ruined if discharged too low).

**TODO**:    
1. Take a look at Battery.msg in the ~/ws/src/pidrone_pkg/msg directory on your drone. This is a custom message we've created to communicate the battery values.
2. In `student_flight_controller_node.py`, do the following:    
    - Fill in each `TODO` regarding the `battery_message` in the `__init__` method.
    - Fill in each `TODO` in the `update_battery_message` method.


## Problem 2: Extracting IMU data
Linear accelerations and attitude (i.e. roll, pitch, yaw) can also be extracted from the FC, thanks to the accelerometer and gyroscope. In addition, the angular rates (e.g. change in roll over change in time) can be calculated by using the attitude measurements.

**TODO**:    
1. Take a look at the [Imu ROS message type](http://docs.ros.org/api/sensor_msgs/html/msg/Imu.html) to get an understanding of the data you'll be collecting.
2. In `student_flight_controller_node.py`, do the following:    
    - Fill in each `TODO` regarding the `imu_message` in the `__init__` method.
    - Fill in each `TODO` in the `update_imu_message` method.
